# LABB practical tutorial - analysis of completion and drop-out

# Introduction

In the practical part we will answer the following questions:

- What is the completion rate in upper secondary education?
- How does it vary with the education chosen at entry in upper secondary education?
- Which types of students have a higher risk of dropping out?

We will do this by defining the pertaining events for a completion rate:

- Entry in upper secondary education
- First certificate in upper secondary education

We will join these two events in order to identify a starting population (all the entrants) and a final population (those who got a first certificate).

For the students who did not complete the education we need to know:

- How many are still in education (right censoring)
- How many left education before 2016 (drop-out).

By this procedure you will understand:

- How to build the base of a (Hierarchical) state file, when you need to build one for your needs
- A better use of the existing Hierarchical state files.



## Setting up the data

We start by loading the simulated data file into R:

In [44]:
# load packages:
library(magrittr) # necessary
library(dplyr)    # necessary
library(tidyr)    # necessary
library(sqldf)    # necessary
library(knitr)    # optional (just for output)
library(kableExtra) # optional (just for output)

# load and transform the data into a dataframe:
d <- readRDS(file = "LongFile_Simulated_2016.rds")
d <- data.frame(d)

In [45]:
dat <- d %>% 
  filter(lernidcat =="NAVS13" & dformflag != "Double student record not selected") %>% 
  mutate(dformflag = factor(dformflag ))

table(dat$lernidcat) 


NAVS13 
442201 

In [46]:
table(dat$dformflag)


                Not applicable Double student record selected 
                         62694                           2702 
         Unique student record 
                        376805 

# Define the events of interest

We are interested in the two events:
- Entry in the upper secondary education
- The first certificate in upper secondary education



## Entry in the upper secondary education

The event is identified as the first record in the upper secondary education:

In [8]:
fsii_entry_ <- dat %>%
              arrange(vn, e_year) %>% # We take the first event of the year
              group_by(vn) %>%
              filter(e_eventType == "Student", # We are interested VET and General education
                     e_educType3 %in% c("Two years VET", 
                                        "Three years VET",
                                        "Four years VET",
                                        "Specialized schools",
                                        "Baccalaureate schools")) %>% 
              select(vn, e_date, e_year, e_educType3, counter) %>%
              mutate(first_record = row_number()==1)

In [9]:
head(fsii_entry_)

vn,e_date,e_year,e_educType3,counter,first_record
1,2011-08-15,2011,Two years VET,352879,TRUE
1,2012-08-15,2012,Two years VET,352914,FALSE
2,2011-08-15,2011,Two years VET,15743,TRUE
2,2012-08-15,2012,Two years VET,15776,FALSE
3,2011-08-15,2011,Two years VET,331977,TRUE
3,2012-08-15,2012,Two years VET,331943,FALSE


We can now select all the first entry in upper secondary:

In [10]:
fsii_entry <- fsii_entry_ %>%
              filter(first_record == TRUE) %>%
              arrange(vn, e_year) %>%
              group_by(vn) %>%
              filter(row_number()==1)

Exploration: You may want to explore the distribution of the entrants:

In [12]:
fsii_entry$e_educType3 <- factor(fsii_entry$e_educType3)
addmargins(table(fsii_entry$e_educType3, fsii_entry$e_year), 1)

,2011
Two years VET,3515
Three years VET,37947
Four years VET,13560
Specialized schools,3641
Baccalaureate schools,19763
Sum,78426


We can see here that we have around 78.000 pupils

## The first certificate in upper secondary education

We can now define the titles. Here we take the first title of the upper secondary degree:

In [13]:
fcert <- dat %>%
  group_by(vn) %>%
  arrange(vn, e_date) %>% # Sort by VN and by date
  filter(as.numeric(e_educType3) %in% c(5:9) # Other way to select education
         & e_gradecompletion == "Completed") %>% # We want only completion records
  select(vn, e_year, e_date, e_educType3, counter) %>% # We select only base variables
  filter(row_number()==1) # We filter the first record

Exploration: When have these titles been obtained?

In [14]:
ta1 <- table(factor(fcert$e_educType3), fcert$e_year)

ta1 

                       
                         2012  2013  2014  2015  2016  2017
  Two years VET            14  2633   242   710    16     0
  Three years VET           0   179 28114  6025  1139     0
  Four years VET            0     0    44 10146  1409   260
  Specialized schools       0     1  2248   940    27     0
  Baccalaureate schools     0     8 14207  3924   141     0

In [15]:
round(addmargins(prop.table(ta1, 1), 2)*100, 2)  

,2012,2013,2014,2015,2016,2017,Sum
Two years VET,0.39,72.84,6.69,19.64,0.44,0.00,100
Three years VET,0.00,0.50,79.29,16.99,3.21,0.00,100
Four years VET,0.00,0.00,0.37,85.56,11.88,2.19,100
Specialized schools,0.00,0.03,69.90,29.23,0.84,0.00,100
Baccalaureate schools,0.00,0.04,77.72,21.47,0.77,0.00,100


## Define the basefile

We can now link the entry to the title

In [18]:
fcert_b <- fcert %>% 
  select(vn, fcert_e_eductype3 = e_educType3, fcert_counter = counter) # Rename variables for entry in upper secondary edu.

fsii_entry_b <- fsii_entry %>%
  select(vn, fsii_e_eductype3 = e_educType3, fsii_counter = counter) # ... the same for the tiltes

# Left join entry - certificates: 
eq <- fsii_entry_b %>% 
  left_join(fcert_b, by = "vn") %>% 
  mutate(
    cert = ifelse(!is.na(fcert_e_eductype3), 1, 0) # A dummy variable telling if certificate
    )

In [19]:
eq %>%
  head() 

vn,fsii_e_eductype3,fsii_counter,fcert_e_eductype3,fcert_counter,cert
1,Two years VET,352879,Two years VET,352397,1
2,Two years VET,15743,Two years VET,15755,1
3,Two years VET,331977,Two years VET,331868,1
4,Two years VET,410742,Two years VET,410654,1
5,Two years VET,182346,Two years VET,182447,1
6,Two years VET,52323,Two years VET,52331,1


We are now able to compute the completion rate:

In [20]:
completion <- c("Not completed", "Completed")
basetab <- addmargins(table(factor(eq$fsii_e_eductype3), factor(eq$cert, labels = completion)), 1)

eqcab <- basetab %>%
  unclass() %>%
  data.frame() 

eqcpr <- round(addmargins(prop.table(basetab, 1)*100, 2), 2) %>%
  unclass() %>%
  data.frame()

cbind(eqcab, eqcpr)

,Not.completed,Completed,Not.completed,Completed,Sum
Two years VET,487,3028,13.85,86.15,100
Three years VET,3336,34611,8.79,91.21,100
Four years VET,1076,12484,7.94,92.06,100
Specialized schools,382,3259,10.49,89.51,100
Baccalaureate schools,718,19045,3.63,96.37,100
Sum,5999,72427,7.65,92.35,100


# Analysis of drop-out

We know how many obtained their first ceritficate.

The second question is : how many are still in education and could potentially obtain the certificate? And how many left?

In order to do this we will:

- select all records for the pupils that did not obtain a certificate.
- find them in the long file


In [21]:
# select all records for the pupils that did not obtain a certificate.
nocert <- eq %>% filter(cert == 0)

# select from the long file all the records that are in the 'nocert' dataset. 
sjps <- semi_join(x = dat, y = nocert, by ="vn")

Let’s look at the trajectories in the long file:

In [22]:
sjps %>% 
  select(vn, e_date, e_educType3, e_gradecompletion) %>% 
  head() 

vn,e_date,e_educType3,e_gradecompletion
3029,2010-08-15,Lower secondary (Extended requirements),Not applicable
3029,2011-08-15,Two years VET,Not applicable
3029,2012-08-15,Two years VET,Not applicable
3029,2013-08-15,Two years VET,Not applicable
3029,2014-08-15,Two years VET,Not applicable
3029,2015-08-15,Two years VET,Not applicable


Just to check: how many pupils did not finish upper secondary?

In [23]:
sjps %>% 
  distinct(vn) %>% 
  nrow()

[1] 5999

Among them who interrupted their education? And who is still in education?

In [25]:
stilledu <- sjps %>%
    select(vn, e_year, e_date, e_gradecompletion, e_educType3, counter) %>%
    group_by(vn) %>%
    arrange(vn, e_year) %>%
    # We select all individuals for whom the last observation was in 2016
    mutate(test = row_number()==n() & e_year == 2016) 

Just to check: How many are still in education?

In [ ]:
stilledu %>% filter(test==TRUE) %>% distinct(vn) %>% nrow()

How many left education:

In [27]:
dropout <- sjps %>%
    select(vn, e_year, e_date, e_gradecompletion, e_educType3, counter) %>%
    group_by(vn) %>%
    arrange(vn, e_year) %>%
    mutate(test = row_number()==n() & e_year < 2016)

dropout %>% filter(test==TRUE) %>% distinct(vn) %>% nrow()

[1] 3899

We have now two datasets:

- A dataset of students who dropped out of education;
- A dataset of students who are still in education (but did not obtain a certificte)

The next step is to prepare a dataset for these two categories of students.

In [29]:
se <- stilledu %>% filter(test==TRUE) %>%  # We select the last record when observed
  select(vn, counter) %>% mutate(finalstatus = 1)

dp <- dropout %>% filter(test==TRUE) %>% 
  select(vn, counter)%>% mutate(finalstatus = 0)

nocert <- bind_rows(se, dp) # Gather the two dataset 

# to check that we have the correct number of people who did not obtained a certificate
dim(nocert)[1]

[1] 5999

By using the counter we are able to join this information to the completion rate file (eq):

In [30]:
eq1 <- eq %>% left_join(nocert, by="vn") 
eq1$gen_counter <- ifelse(!is.na(eq1$counter), eq1$counter, eq1$fcert_counter)

We can verify that everything went well by crosstabulating the two variables:

In [34]:
completion <- c("Not completed", "Completed")
finstat <- c("Left edu. < 2016", "In edu. 2016")

table(factor(eq1$cert, labels = completion),
      factor(eq1$finalstatus, labels=finstat), useNA = "always")

               
                Left edu. < 2016 In edu. 2016  <NA>
  Not completed             3899         2100     0
  Completed                    0            0 72427
  <NA>                         0            0     0

In [35]:
cert_det <- c("Left edu. < 2016", "In edu. 2016", "Completed")
# if finalstatus == 0 (drop-out) and cert == 0 (Nocertificate) --> 0
eq1$cert_det <- ifelse(eq1$cert == 0 & eq1$finalstatus == 0, 0, 
                       # if finalstatus == 1 (in Edu) and cert == 0 (Nocertificate) --> 1
                       ifelse(eq1$cert == 0 & eq1$finalstatus == 1, 1,
                              # if cert == 1 (Certificate obtained) --> 2
                              ifelse(eq1$cert == 1, 2, 9999 # Trash code to detect errors
                                     )
                              )
                       )

# Assign labels to the variable:
eq1$cert_det <- factor(eq1$cert_det, labels = cert_det)

# Just a tabulation:
table(eq1$cert_det, factor(eq1$cert, labels = completion)) 

                  
                   Not completed Completed
  Left edu. < 2016          3899         0
  In edu. 2016              2100         0
  Completed                    0     72427

We now have a more complete information on the paths in upper secondary education:

In [36]:
eqcomple <- addmargins(table(factor(eq1$fsii_e_eductype3),eq1$cert_det), 1)

eqcab <- eqcomple %>%
  unclass() %>%
  data.frame() 

eqcpr <- round(addmargins(prop.table(eqcomple, 1)*100, 2), 2) %>%
  unclass() %>%
  data.frame()

cbind(eqcab, eqcpr) 

,Left.edu....2016,In.edu..2016,Completed,Left.edu....2016,In.edu..2016,Completed,Sum
Two years VET,418,69,3028,11.89,1.96,86.15,100
Three years VET,2375,961,34611,6.26,2.53,91.21,100
Four years VET,614,462,12484,4.53,3.41,92.06,100
Specialized schools,151,231,3259,4.15,6.34,89.51,100
Baccalaureate schools,341,377,19045,1.73,1.91,96.37,100
Sum,3899,2100,72427,4.97,2.68,92.35,100


In [37]:
saveRDS(eq1, file = "erfolg_quote.rds")

# Other sequence mining

How many records per individual?

In [38]:
dd <- d %>% 
  group_by(vn) %>%
  mutate(number = n()) %>%
  select(vn, e_date, e_gradecompletion, e_educType3, e_educYear, number)

head(dd, n = 10) 

vn,e_date,e_gradecompletion,e_educType3,e_educYear,number
1,2011-08-15,Not applicable,Two years VET,First year,3
1,2012-08-15,Not applicable,Two years VET,Second year,3
1,2013-06-30,Completed,Two years VET,Not applicable,3
2,2010-08-15,Not applicable,Lower secondary (Extended requirements),11th year HarmoS,4
2,2011-08-15,Not applicable,Two years VET,First year,4
2,2012-08-15,Not applicable,Two years VET,Second year,4
2,2013-06-30,Completed,Two years VET,Not applicable,4
3,2010-08-15,Not applicable,Lower secondary (Extended requirements),11th year HarmoS,4
3,2011-08-15,Not applicable,Two years VET,First year,4
3,2012-08-15,Not applicable,Two years VET,Second year,4


You can self join the long file with himself in order to find the sequences of students who reoriented, repeated or exited education:

In [40]:
dat_1 <- dat %>%
          arrange(vn, e_date) %>%
          group_by(vn) %>%
          mutate(Count_rw = 1:n())

# Generate an internal counter for the record:
head(dat_1[, c("vn", "e_date", "Count_rw")])

vn,e_date,Count_rw
1,2011-08-15,1
1,2012-08-15,2
1,2013-06-30,3
2,2010-08-15,1
2,2011-08-15,2
2,2012-08-15,3


In [41]:
dat_2 <- sqldf(
            "SELECT s.vn, s.e_year, s.e_gradecompletion, s.e_educType3,s.Count_rw AS Count_s,
              t.e_year AS e_year_b, t.e_educType3 AS e_educType3_b, t.e_educYear AS e_educYear_b, 
              t.Count_rw AS Count_t
              FROM dat_1 AS s LEFT JOIN dat_1 AS t
              ON s.vn = t.vn AND 
              t.Count_rw - s.Count_rw == 1"
            ) 

head(dat_2) 

vn,e_year,e_gradecompletion,e_educType3,Count_s,e_year_b,e_educType3_b,e_educYear_b,Count_t
1,2011,Not applicable,Two years VET,1,2012,Two years VET,Second year,2
1,2012,Not applicable,Two years VET,2,2013,Two years VET,Not applicable,3
1,2013,Completed,Two years VET,3,NA,NA,NA,NA
2,2010,Not applicable,Lower secondary (Extended requirements),1,2011,Two years VET,First year,2
2,2011,Not applicable,Two years VET,2,2012,Two years VET,Second year,3
2,2012,Not applicable,Two years VET,3,2013,Two years VET,Not applicable,4


In [42]:
dout <- sqldf(
            "SELECT s.vn, s.e_date, s.e_year, s.e_gradecompletion, s.e_educYear, s.e_educType3 FROM dat AS s
                WHERE vn IN (SELECT vn FROM dat_2 WHERE e_year_b - e_year > 1) 
                ORDER BY vn, e_date"
            )

head(dout, n = 10)

vn,e_date,e_year,e_gradecompletion,e_educYear,e_educType3
2850,2010-08-15,2010,Not applicable,Undefined,Special needs education
2850,2011-08-15,2011,Not applicable,First year,Two years VET
2850,2013-08-15,2013,Not applicable,Third year,Two years VET
2850,2014-06-30,2014,Completed,Not applicable,Two years VET
2851,2010-08-15,2010,Not applicable,First year,Transitional options
2851,2011-08-15,2011,Not applicable,First year,Two years VET
2851,2014-06-30,2014,Completed,Not applicable,Two years VET
2852,2010-08-15,2010,Not applicable,11th year HarmoS,Lower secondary (Extended requirements)
2852,2011-08-15,2011,Not applicable,First year,Two years VET
2852,2012-08-15,2012,Not applicable,Second year,Two years VET
